In [6]:
from os import path
import string
import sys
sys.path.append("..")

import pandas as pd

from src.Dataset import Dataset
from src.visualisations import *
from src.data_processing import *
from src.model import *

In [7]:
train_set = path.join('..', 'data', 'training_set.json')
test_set = path.join('..', 'data', 'testing_set.json')
data = Dataset(train_set, test_set)

Loaded 7100 rows


# Création d'un nouveau modèle
## Traitement des données
### Traitement du déséquilibre interclasses
Avant toute chose, il est nécessaire de résoudre le problème du déséquilibre entre le nombre de phrases dans chaque classe.
Pour cela, nous avons décidé de simplement retirer un certain nombre de phrases (aléatoirement) de la classe "irrelevant" afin que toutes les classes aient en moyenne à peu près le même nombre de phrases que les autres classes

Peut-être :
- Tout mettre au meme niveau
- Data augmentation

In [8]:
balanced_data = balance(data).reset_index()

AttributeError: 'tuple' object has no attribute 'reset_index'

### Préparation pour l'entrainement
#### Vectorisation
On vectorise tout

In [ ]:
vectorized_sentences = vectorize_data(balanced_data['sentence'])

#### One hot encodage

In [ ]:
labels, one_hot_labels = to_one_hot(balanced_data['intent'])

## Le modèle

In [ ]:
model = NlpModel(vectorized_sentences.shape[1])
model.summary()

On redécoupe les données en en gardant 20% pour tester le modèle

In [ ]:
sentences_train, sentences_test, y_train, y_test = cut_data(vectorized_sentences, one_hot_labels, 0.2)

On entraine le modèle

In [ ]:
h = model.train(sentences_train, y_train, sentences_test, y_test, False)

## Evaluations
Evaluons le modèle pour le comparer avec le modèle que nous avions précédemment

In [ ]:
m = model._model.evaluate(sentences_test, y_test)

In [ ]:
predic_prob = model.predict(sentences_test)
predictions = np.argmax(y_test, axis=1)

In [ ]:
metrics_analysis(predictions, np.argmax(y_test, axis=1))

In [ ]:

"""Compute and display roc curve for each class

Parameters:
predictions_prob (array): Table of predictions (must contain probability for each class)

"""
classes = ['find-train', 'irrelevant', 'find-flight', 'find-restaurant',
           'purchase', 'find-around-me', 'provide-showtimes', 'find-hotel']



fpr = {}
tpr = {}
auc = {}
thresh = {}

for cl in range(8):
    fpr[cl], tpr[cl], thresh[cl] = roc_curve(np.argmax(y_test, axis=1)==cl, predic_prob[:, cl])

    y_pred = predictions == cl
    y_true = np.argmax(y_test, axis=1) == cl
    auc[cl] = roc_auc_score(y_true, y_pred)

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

for i in range(len(classes)):
    cl = i
    name = f"{cl} (AUC={auc[cl]:.2f})"
    fig.add_trace(go.Scatter(x=fpr[cl], y=tpr[cl], name=name, mode='lines'))

fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=500
)
fig.show()

## Sauvegarde
Pour finir, on sauvegarde le modèle afin de l'utiliser dans une API

In [ ]:
tf.keras.models.save_model(model._model, filepath='../models/model_11_12_2020.hdf5', save_format='h5')